In [1]:
import sys

import docx
import os
from datetime import datetime

import pandas as pd
import pymysql
from pymysql import MySQLError

def get_file_info(file_path):
    file_size = os.path.getsize(file_path)                  # 파일 크기를 바이트 단위로 가져오기 
    file_creation_time = os.path.getctime(file_path)        # 파일 생성일자를 가져오기
    file_modification_time = os.path.getmtime(file_path)    # 파일 최종 수정일자를 가져오기
    # file_name = os.path.basename(file_path)                 # 파일명 추출
    file_name, file_extension = os.path.splitext(os.path.basename(file_path))                 # 확장명 뺀 파일명 추출
    
    return file_name, file_size, file_creation_time, file_modification_time

def format_time(timestamp):
    dt_object = datetime.fromtimestamp(timestamp)               # 에포크 시간을 datetime 객체로 변환
    formatted_time = dt_object.strftime('%Y-%m-%d %H:%M:%S')    # 원하는 형식으로 시간을 포맷팅
    
    return formatted_time

def count_words_docx(file_path):
    total_words = 0
    doc = docx.Document(file_path)

    for paragraph in doc.paragraphs:
        total_words += len(paragraph.text.split())   # 문서 내 단어 수 세기 

    return total_words

In [2]:
# MySQL 연결 정보
mysql_host = '172.30.1.36'
mysql_port = 13333
mysql_user = 'vnc'
mysql_password = 'vnc'
mysql_database = 'vnc'


excel_tb = 'load_vnc_org_lst_sim_20231130_05'
file_tb = 'load_vnc_file_list_sim_20231130_05'
docx_tb = 'load_vnc_file_docx_sim_20231130_05'

In [7]:
# org 정보
JOB_YMD = '20231130'
WORKER_ID = 'CW055'
PUB_YMD = ''
WTR_KR = '구텐베르크 프로젝트'
WTR_VN = 'Dự án Gutenberg'
CTR_KR = ''
CTR_VN = ''
ORG_TYP = 'html'
DAT_TYP = '문학'
TOPIC_KR = '문화/역사/예술'
TOPIC_VN = 'VĂN HÓA/ LỊCH SỬ/ NGHỆ THUẬT'
TITLE = ''
DOC_STYLE = '구어체'
TOPIC_CD = '05'


In [ ]:
# 텍스트 파일 경로
# root_path = 'Y:/_구어체_학교이야기/'                             # CW047 20231114 SEQ 1 ~ 54
# root_path = 'Y:/_구어체_아동소설/'                                # CW047 20231114 SEQ 55 ~ 239 
# root_path = 'Y:\\구어체_모음/2023-11-14 구어체\\자동화2/'        # CW047 20231114 SEQ 240 ~ 809
# root_path = 'Y:\\구어체_모음\\2023-11-14 구어체\\자동화2_추가/'     # CW047 20231114 SEQ 810 ~ 1032
# root_path = f'Y:\\구어체_모음\\2023-11-22 구어체\\자동화3_CW055/'     # CW055 20231122 SEQ 1 ~ 131
# root_path = f'Y:\\구어체_모음\\2023-11-22 구어체\\자동화4_CW056/'     # CW056 20231122 SEQ 1 ~ 1032
# root_path = f'Y:\\구어체_모음\\2023-11-28 구어체\\자동화_en_CW055/'     # CW056 20231128 SEQ 1 ~ ?
root_path = r'Y:\구어체_모음\2023-11-30 구어체\자동화27_CW055/'     # CW055 20231130 SEQ 1 ~ 182

file_list = os.listdir(root_path)
file_list = [file for file in file_list if not file.split('.')[1] == 'xlsx' and len(file) > 10]

# file = 'httpswww.gutenberg.orgcacheepub952pg952.txt'

try:
    db = pymysql.connect(host=mysql_host,port=mysql_port,user=mysql_user,passwd=mysql_password,db=mysql_database)
    cursor = db.cursor(pymysql.cursors.DictCursor)
    print("Connected to MySQL database")
    
    # 세이브포인트
    sql = ''
    sql = 'SAVEPOINT a'
    cursor.execute(sql)
                
    for idx, file in enumerate(file_list, start=0):
    # if True:
        
        # FILE_NAME을 위한 SEQ
        ORG_SEQ = idx + 1
        # ORG_SEQ = 1
        FILE_NAME = TOPIC_CD + '_' + WORKER_ID + '_' + JOB_YMD + '_' + str(ORG_SEQ).rjust(4, '0')
        
        # print(f"{file} : {FILE_NAME}")
        
        # FILE_LIST에 들어갈 정보
        txt_name, FILE_SIZE, file_creation_time, file_modification_time = get_file_info(os.path.join(root_path, file))
        
        # 시간을 원하는 형식으로 변환
        FILE_CREATETIME = format_time(file_creation_time)
        FILE_CHANG_TIME = format_time(file_modification_time)

        DAT_SRC = txt_name
        DAT_SRC = DAT_SRC.replace("'", "''")
        
        docx_bulk_insert_datas = []
        chunk_size = 300
        
        # 파일 열기 (기본적으로는 읽기 모드인 'r' 모드로 열림)
        with open(os.path.join(root_path, file), 'r', encoding='utf-8') as txt:
            
            # 파일의 내용 읽기
            # file_content = txt.read()
            
            # paragraphs = txt.read().split('\n\n')  # 빈 줄을 기준으로 문단 나누기

            lines = txt.read().split('\n')
            
            # 빈 문장 제거
            lines = [line for line in lines if not line == '' ]           # 빈공백으로만 있는 텍스트 제거
            lines = [line for line in lines if not line.strip() == '' ]
            
            start_index = 0
            end_index = 0
            
            # 한줄 씩 순회
            for i, line in enumerate(lines, start=0):
                
                if str(line).startswith('Tiêu đề:'):
                    TITLE = str(line)[9:].strip()
                    TITLE = TITLE.replace("'", "''")
                
                if str(line).startswith('Tác giả:'):
                    CTR_KR = str(line)[9:].strip()
                    CTR_KR = CTR_KR.replace("'", "''")
                    CTR_VN = CTR_KR
                    
                if str(line).startswith('Ngày phát hành:'):
                    PUB_YMD = str(line)[16:].strip()
                    # PUB_YMD = str(line).split()[8] + str(line).split()[6].rjust(2, '0') + str(line).split()[4].rjust(2, '0')
                
                if str(line).startswith('*** Bắt đầu'):
                # if str(line).startswith('Chương i'):
                    start_index = i
                
                if str(line).startswith('*** Kết thúc'):
                    end_index = i
                    
            # print(f"{PUB_YMD}, {TITLE}, {CTR_KR}")
            # print(f"{start_index}, {end_index}")
            WORD_COUNT = 0
            DOCX_SEQ = 1
            
            lines = lines[start_index + 3 : end_index - 1]  
            
            # 각 문단에 대한 처리 수행
            for i, line in enumerate(lines):
                
                # if i == end_index:
                #     break
                        
                WORD_COUNT += len(line.split(' '))   # 문서 내 단어 수 세기 
                
                # '\n' 별로 저장할 때           
                line = line.replace("'", "''")
                line = line.replace("\u00A0", " ")    # strip()이 걸러내지 못하는 &nbsp 제거
                line = line.replace("\uFEFF", " ")    # HTML 코드(&#65279;) 제거
                line = line.replace('\t', ' ')
                line = line.replace('\xa0', ' ')
                line = line.strip()     # 앞 뒤 공백 제거
                # print(f'{DOCX_SEQ}번, {WORD_COUNT} : {line} \n')
                    
                # 벌크에 넣기
                docx_bulk_insert_datas.append((FILE_NAME, DOCX_SEQ, line))

                # text line_seq 증가
                DOCX_SEQ += 1   
          
        # org 테이블
        sql = ""
        sql = sql + f" INSERT INTO {excel_tb} "
        sql = sql + " (COL_PK, SEQ, WORKER_ID, JOB_YMD, PUB_YMD, WTR_KR, WTR_VN, CTR_KR, CTR_VN, ORG_TYP, DAT_SRC, DAT_TYP, TOPIC_KR, TOPIC_VN, TITLE, FILE_NAME, DOC_STYLE) "
        sql = sql + f''' VALUES ('{FILE_NAME}', '{ORG_SEQ}', '{WORKER_ID}', '{JOB_YMD}', '{PUB_YMD}', '{WTR_KR}', 
                                '{WTR_VN}', '{CTR_KR}', '{CTR_VN}', '{ORG_TYP}', '{DAT_SRC}', '{DAT_TYP}', '{TOPIC_KR}', '{TOPIC_VN}', '{TITLE}', '{FILE_NAME}', '{DOC_STYLE}') '''
        cursor.execute(sql)
        # db.commit()
        
        # file_list 테이블
        sql = ""
        sql =  sql + f" INSERT INTO {file_tb} "
        sql = sql + " (workymd, worker_id, workcnt, file_name, file_size, file_createtime, file_chang_time, word_count, org_file_name) "
        sql = sql + f""" VALUES ('{JOB_YMD}', '{WORKER_ID}', '55' , '{FILE_NAME}', '{FILE_SIZE}', '{FILE_CREATETIME}', '{FILE_CHANG_TIME}', '{WORD_COUNT}', '{root_path}{file}') """
        cursor.execute(sql)
        # db.commit()
        
        # docx 테이블
        for i in range(0, len(docx_bulk_insert_datas), chunk_size):
            batch = docx_bulk_insert_datas[i:i+chunk_size]
            # print(batch)
            
            sql = ""
            sql = sql + f" INSERT INTO {docx_tb} "
            sql = sql + " (FILE_NAME, SEQ, FILE_TXT) "
            sql = sql + " VALUES (%s, %s, %s)"
            cursor.executemany(sql, batch)
            # db.commit()
            # print(f'{len(batch)}개 저장완료')   
    
        print(f"{file} : {FILE_NAME} 처리.") 
    
    db.commit()

except MySQLError as e:
    print(f"MySQLError at {file} : {FILE_NAME}")
    print(f"MySQLError: {e}")
    
    sql = ''
    sql = 'rollback to a'
    cursor.execute(sql)
    
    # sql = ''
    # sql = sql + f'DELETE FROM {excel_tb} WHERE WORKER_ID = "{WORKER_ID}" AND JOB_YMD = "{JOB_YMD}"'
    # cursor.execute(sql)
    # db.commit()
    
    # sql = ''
    # sql = sql + f'DELETE FROM {file_tb} WHERE WORKER_ID = "{WORKER_ID}" AND WORKYMD = "{JOB_YMD}"'
    # cursor.execute(sql)
    # db.commit()
    
    # sql = ''
    # sql = sql + f'DELETE FROM {docx_tb} WHERE FILE_NAME LIKE "%{WORKER_ID}_{JOB_YMD}%"'
    # cursor.execute(sql)
    # db.commit()
    sys.exit()

except Exception as e:
    print(f"Exception at {file} : {FILE_NAME}")
    print(f"Exception: {e}")
    sys.exit()
    
finally:
    if cursor:
        cursor.close()
            
    if db:
        db.close()
        print("Disconnected from MySQL database")


